# Data preparation

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
!cp drive/MyDrive/qa_data.jsonl.zip .

In [25]:
!unzip qa_data.jsonl.zip

Archive:  qa_data.jsonl.zip
  inflating: qa_data.jsonl           
  inflating: __MACOSX/._qa_data.jsonl  


In [30]:
!head -n 200000 qa_data.jsonl > train.jsonl
!head -n 250000 qa_data.jsonl | tail -n 50000 > valid.jsonl
!head -n 300000 qa_data.jsonl | tail -n 50000 > test.jsonl

# Imports

In [ ]:
import pandas as pd
import json

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

In [42]:
from torchtext.legacy.data import BucketIterator

In [ ]:
START_TOKEN = "<bos>",
END_TOKEN = "<eos>"